In [46]:
# conda activate anndata

import os
import sys
import anndata as ad

sys.path.append("/mnt/lareaulab/reliscu/code")

from junction2psi import *

In [ ]:
adata = ad.read_h5ad("/mnt/lareaulab/reliscu/projects/NSF_GRFP/data/bulk/hahn_2023/cortex/hahn_2023_cortex_STAR_SJ_counts.h5ad")

In [48]:
adata.shape

(59, 157406)

In [49]:
SJ_counts_table = pd.DataFrame(adata.X.T, columns=adata.obs_names, index=adata.var_names)

In [50]:
SJ_counts_table.shape

(157406, 59)

In [51]:
events_i1 = pd.Index([x[:-3] for x in SJ_counts_table.index if '_I1' in x])
events_i2 = pd.Index([x[:-3] for x in SJ_counts_table.index if '_I2' in x])
events_se = pd.Index([x[:-3] for x in SJ_counts_table.index if '_SE' in x])

events = events_i1.intersection(events_i2).intersection(events_se)
i1_events = [x + '_I1' for x in events]
I1_table = SJ_counts_table.loc[i1_events]
I1_table.index = events

i2_events = [x + '_I2' for x in events]
I2_table = SJ_counts_table.loc[i2_events]
I2_table.index = events

se_events = [x + '_SE' for x in events]
SE_table = SJ_counts_table.loc[se_events]
SE_table.index = events

I1_filt = I1_table.index[I1_table.sum(axis=1) > 0]
I2_filt = I2_table.index[I2_table.sum(axis=1) > 0]
SE_filt = SE_table.index[SE_table.sum(axis=1) > 0]
filtered_events = I1_filt.intersection(I2_filt).intersection(SE_filt)

I1_table = I1_table.loc[filtered_events]
I2_table = I2_table.loc[filtered_events]
SE_table = SE_table.loc[filtered_events]

psi = ((I1_table + I2_table) /(2*SE_table + I1_table + I2_table)).fillna(0)
reads = SE_table + I1_table + I2_table

In [52]:
psi.shape[0]

13300

In [ ]:
psi.to_csv(f"data/hahn_2023_cortex_STAR_STAR_exon_PSI.csv")
reads.to_csv(f"data/hahn_2023_cortex_STAR_STAR_exon_counts.csv")